<a href="https://colab.research.google.com/github/kaspergroenbek98/first-first-year-project/blob/master/SklearnNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/kaspergroenbek98/first-first-year-project.git


Cloning into 'first-first-year-project'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 81 (delta 35), reused 23 (delta 8), pack-reused 0
Unpacking objects: 100% (81/81), done.


In [28]:
import numpy as np
import nltk
import csv
import re
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
# Using pandas to load the data file into a CSV.
# ISO-8859-1 is the encoding for tweets.
df = pd.read_csv("first-first-year-project/data.csv", encoding = "ISO-8859-1")
#First step to cleaning the data, removing the categories we dont need.
first_clean = df.drop(["other_topic","resolution_topics", "tweet_coord", "tweet_created","tweet_id", "name", "retweet_count", "tweet_date", "user_timezone"], axis = 1)
first_clean


,gender,Resolution_Category,text,tweet_location,tweet_state,tweet_region
0,female,Health & Fitness,"#NewYearsResolution :: Read more books, No scr...",Southern California,CA,West
1,female,Humor,#NewYearsResolution Finally master @ZJ10 's pa...,New Jersey,NJ,Northeast
2,male,Personal Growth,#NewYearsResolution to stop being so damn perf...,Hollywood,CA,West
3,male,Philanthropic,My #NewYearsResolution is to help my disabled ...,Metro NYC,NY,Northeast
4,female,Personal Growth,#NewYearsResolution #2015Goals #2015bucketlist...,"Pittsburgh, Pennsylvania",PA,Northeast
...,...,...,...,...,...,...
5006,female,Recreation & Leisure,Tomorrow I start @JustifiedFX because @natalie...,NC/TN,TN,South
5007,female,Humor,"holy crap, people. EYES OPEN WHEN DRIVING. #Ne...","charleston, nyc",NY,Northeast
5008,female,Humor,RT @moJO_SHabby: Start parody of her blog #NYR...,Memphis,TN,South
5009,female,Career,RT @kscmaghirang: To have an excellent job bef...,Paris USA,TX,South


In [3]:
#Create a list of ALL tweets. (Each element in the list is a tweet)
sequence = list()
for i in range(len(OGdata[:,6])):
  sequence.append(OGdata[i,6].strip())

#Using Sklearn, playing around with the CountVectorizer.
vectorizer = CountVectorizer(analyzer="word", lowercase=True, stop_words= "english",min_df= 5, binary= True)
X = vectorizer.fit_transform(sequence)
X.toarray()
print(vectorizer.get_feature_names())
print(vectorizer.vocabulary_.get(u"newyearsresolution"))
print(X.toarray())
print(X.shape)

['000', '10', '100', '1024', '1080p', '11', '12', '13', '14', '15', '17', '18', '1920x1080', '1st', '20', '200', '2013', '2014', '2015', '2015goals', '2016', '21', '24', '2k15', '2nd', '30', '365', '40', '50', '5sos', '__ù', '_ù', '_ù_ä', '_ùª', '_ùªî', '_ùó', '_ùô', '_ùô_', '_ùôî', '_ùõ', '_ùõø', 'able', 'abs', 'accept', 'accomplish', 'accomplished', 'account', 'achieve', 'act', 'acting', 'action', 'active', 'actually', 'add', 'adult', 'adventure', 'advice', 'afraid', 'age', 'ago', 'ahead', 'ain', 'alcohol', 'allow', 'alot', 'amazing', 'amp', 'amwriting', 'anybody', 'anymore', 'apartment', 'app', 'appreciate', 'arms', 'art', 'asap', 'asapferg', 'ask', 'asked', 'asking', 'ass', 'asshole', 'attend', 'attention', 'attitude', 'auntieannes', 'avoid', 'away', 'awesome', 'baby', 'bacon', 'bad', 'bae', 'bag', 'ball', 'balls', 'band', 'bar', 'baseball', 'based', 'battle', 'bc', 'beard', 'beat', 'beautiful', 'bed', 'beer', 'begin', 'believe', 'best', 'better', 'bible', 'big', 'bigger', 'birthda

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X)
X_train_tfidf.shape


(5011, 996)

In [7]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, )

NameError: ignored